In [1]:
import mcl
mcl.approx_pi(1000)

3.1406380562059946

In [4]:
import pyzx as zx
import json

# Allows us to visualize circuits from qasm
from cirq.contrib.qasm_import import circuit_from_qasm


ocaml_output = """
[
  { "Prep": { "on_qubits": [ 0 ] } },
  { "Prep": { "on_qubits": [ 1 ] } },
  { "Prep": { "on_qubits": [ 2 ] } },
  { "XCorrect": { "on_qubits": [ 1 ], "signals": [] } }
]
"""

#{ "Entangle": { "on_qubits": [ 0, 1 ] } }
measure_string = """
  {
    "Measure": {
      "on_qubits": [ 0 ],
      "angle": 0.0,
      "signal_s": [],
      "signal_t": []
    }
  }
"""

# loaded_program = json.loads(ocaml_output)

# from mcl import ZXBuilder
# zx_graph = ZXBuilder(loaded_program)
# qasm = zx_graph.to_qasm()
# zx.draw(zx_graph.to_circuit())
# zx.draw(zx_graph.graph)
# print(qasm)
# circuit = circuit_from_qasm(qasm)

# print(circuit)

# print('\n')

# from mcl import CirqBuilder
# circuit = CirqBuilder(loaded_program)
# qasm = circuit.to_qasm()
# print(qasm)
# print(circuit.to_circuit())

In [14]:

teleportation_output = """
[
    {"Prep": {"on_qubits": [1]}}, 
    {"Prep": {"on_qubits": [2]}}, 
    {"Prep": {"on_qubits": [3]}}, 
    {"Entangle": {"on_qubits": [1, 2]}}, 
    {"Measure": {"on_qubits": [1], "angle": 0.0, "signal_s": [], "signal_t": []}}, 
    {"XCorrect": {"on_qubits": [2], "signals": [1]}}, 
    {"Entangle": {"on_qubits": [2, 3]}}, 
    {"Measure": {"on_qubits": [2], "angle": 0.0, "signal_s": [], "signal_t": []}}, 
    {"XCorrect": {"on_qubits": [3], "signals": [2]}}
]
"""
from mcl import StrictCirqBuilder, ValidCirqBuilder
loaded_program = json.loads(teleportation_output)

strict_circuit = StrictCirqBuilder(loaded_program)
print(strict_circuit.to_circuit())
print(strict_circuit.to_qasm())

valid_circuit = ValidCirqBuilder(strict_circuit.to_circuit())
valid_circuit.rearrange_measurements()

print(valid_circuit.to_circuit())
print(valid_circuit.stripped_circuit)

qasm = valid_circuit.to_qasm()

# Get first circuit
zx_circuit = zx.qasm(qasm)
zx.draw(zx_circuit)

# Optimize using PyZX
zx_graph = zx_circuit.to_graph()
zx.full_reduce(zx_graph, quiet=False)
zx_graph.normalize()
zx.draw(zx_graph)

# Convert back to circuit
optimized_circuit = zx.extract_circuit(zx_graph.copy())
zx.draw(optimized_circuit)
print(optimized_circuit.to_qasm())

print(zx.compare_tensors(zx_circuit, optimized_circuit))

1: ───H───@───M───@───────────────
          │       │
2: ───H───@───────X───@───M───@───
                      │       │
3: ───H───────────────@───────X───
// Generated from Cirq v0.12.0

OPENQASM 2.0;
include "qelib1.inc";


// Qubits: [1, 2, 3]
qreg q[3];
creg m_1[1];
creg m_2[1];


h q[0];
h q[1];
h q[2];
cz q[0],q[1];
measure q[0] -> m_1[0];
cx q[0],q[1];
cz q[1],q[2];
measure q[1] -> m_2[0];
cx q[1],q[2];

1: ───H───@───────@───M───────────────
          │       │
2: ───H───@───────X───@───────@───M───
                      │       │
3: ───H───────────────@───────X───────
1: ───H───@───────@───────────────
          │       │
2: ───H───@───────X───@───────@───
                      │       │
3: ───H───────────────@───────X───


spider_simp: 2. 2. 1.  3 iterations
pivot_simp: 1.  1 iterations


OPENQASM 2.0;
include "qelib1.inc";
qreg q[3];
h q[2];
h q[0];
cz q[0], q[1];
h q[1];
cz q[1], q[2];
cz q[0], q[1];
rz(1.0*pi) q[0];
h q[2];
h q[0];
cz q[1], q[2];
h q[2];
h q[0];

True
